In [6]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import joblib
import optuna
import json
import sys

from sklearn.metrics import f1_score

# gradient boosting libraries
import xgboost as xgb
from catboost import CatBoostClassifier

# --- TabM stuff ---
from f2_preprocessor import Preprocessor as F2Preprocessor
from TabM_save_load_new import TabM_save_load   # your class file with save/load logic
from tabm import TabM
from rtdl_num_embeddings import LinearReLUEmbeddings

# --- CatBoost preprocessor (cc5) ---
from cc5_preprocessor import Preprocessor as CC5Preprocessor

# --- Robust Preprocessor (SW_preprocessor) ---
from SW_preprocessor import enhanced_feature_engineer, RobustPreprocessor

device = 'cpu'

In [7]:
df = pd.read_csv("data/Training_TriGuard.csv")
df = df.dropna(subset=['subrogation'])
target_col = "subrogation"
y_train = df[target_col].astype(int).to_numpy()

real_test = pd.read_csv("data/Testing_TriGuard.csv")

In [8]:
def build_model(n_num_features: int,
                cat_cardinalities,
                trial: optuna.Trial) -> nn.Module:
    # EDIT: tighten but beef up architecture search space
    n_blocks = trial.suggest_int("n_blocks", 3, 5)                 # was 2–5
    d_block  = trial.suggest_int("d_block", 384, 1024, log=True)   # was 256–1024
    dropout  = trial.suggest_float("dropout", 0.0, 0.4)
    k        = trial.suggest_int("k", 8, 32, step=8)               # keep 8–32

    num_emb = LinearReLUEmbeddings(n_num_features)

    model = TabM.make(
        n_num_features=n_num_features,
        num_embeddings=num_emb,
        cat_cardinalities=cat_cardinalities,
        d_out=1,
        n_blocks=n_blocks,
        d_block=d_block,
        dropout=dropout,
        k=k,
    )

    return model.to(device)

## Loading TabM

In [9]:
tabm_pipeline = TabM_save_load.load(
    path="models/tabm_full_pipeline_5938",   # <-- adjust if needed
    build_model_fn=build_model,
    device=device,
)

def tabm_predict_proba(df_input: pd.DataFrame) -> np.ndarray:
    """Use the saved TabM pipeline (with f2_preprocessor) to get P(y=1)."""
    probs, _ = tabm_pipeline.predict(df_input)
    return probs

## Loading XGBoost

In [10]:
sys.modules["__main__"].RobustPreprocessor = RobustPreprocessor

xgb_assets = joblib.load("models/TriGuard_SW14_0.606.pkl")

xgb_preproc           = xgb_assets["preprocessor"]       # can be a function or a transformer
xgb_feature_selector  = xgb_assets["feature_selector"]
xgb_selected_features = xgb_assets["selected_features"]
xgb_model             = xgb_assets["model"]
xgb_best_threshold    = xgb_assets["threshold"]          # we'll ignore this for ensemble


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.6.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator SelectFromModel from version 1.6.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [11]:
def xgb_predict_proba(df_input: pd.DataFrame) -> np.ndarray:
    """
    Apply the XGBoost preprocessing function/object + feature selector + model.
    """
    # If xgb_preproc is a transformer (Pipeline / ColumnTransformer):
    #   X_proc = xgb_preproc.transform(df_input)
    # If it's a function:
    #   X_proc = xgb_preproc(df_input)
    try:
        X_proc = xgb_preproc.transform(df_input)
    except AttributeError:
        X_proc = xgb_preproc(df_input)

    X_sel  = xgb_feature_selector.transform(X_proc)
    proba  = xgb_model.predict_proba(X_sel)[:, 1]
    return proba


## Loading CatBoost

In [27]:
target_col = "subrogation"

# 1. Prepare X, y exactly as in the original notebook
# (You already have df loaded with subrogation.)
df = df.dropna(subset=[target_col])  # just in case

X = df.drop(columns=[target_col]).copy()
y = df[target_col].astype(int).copy()

# 2. Rebuild the original preprocessor in CATBOOST mode
pre = CC5Preprocessor(smoothing_factor=5, mode="catboost")
pre.fit(X, y)

# 3. Transform the FULL training set to get the canonical feature matrix
X_proc = pre.transform(X)

# These are the exact feature columns CatBoost should see
cb_train_cols = X_proc.columns.tolist()

print("Rebuilt CatBoost preprocessor in 'catboost' mode.")
print("Processed train shape:", X_proc.shape)
print("First few columns:", cb_train_cols[:10])

# 4. Load the saved CatBoost model
cb_model = CatBoostClassifier()
cb_model.load_model("models/catboost_mod_f1_0.5937_cv.cbm")  # <-- use your real file name/path


Fitting Preprocessor in 'catboost' mode...
CatBoost mode: Skipping target encoding learning.
Learning statistical parameters for Z-scoring...
Fit complete.
Transforming data in 'catboost' mode...
Applying learned statistical transforms (Z-scores)...
CatBoost mode: Skipping target encoding application.
CatBoost mode: Dropping unused object/datetime columns...
Dropping: ['witness_present_ind', 'claim_date']
Transform complete.
Rebuilt CatBoost preprocessor in 'catboost' mode.
Processed train shape: (17999, 161)
First few columns: ['claim_number', 'year_of_born', 'gender', 'email_or_tel_available', 'safety_rating', 'annual_income', 'high_education_ind', 'address_change_ind', 'living_status', 'zip_code']


In [30]:
def cb_predict_proba(df_input: pd.DataFrame) -> np.ndarray:
    """
    Use the original cc5 Preprocessor (catboost mode) + CatBoost model
    to get P(y=1) for any input df (train or test).

    - Drop target column if present.
    - Apply pre.transform (same FE + encoding as training).
    - Reindex to cb_train_cols (fill missing with 0).
    - Call cb_model.predict_proba(...).
    """
    # Drop target if it sneaks in
    X_raw = df_input.drop(columns=[target_col], errors="ignore").copy()

    # Apply the same feature engineering + preprocessing
    X_cb = pre.transform(X_raw)

    # Align columns to what CatBoost was trained on
    X_cb = X_cb.reindex(columns=cb_train_cols, fill_value=0)

    # Predict probabilities
    proba = cb_model.predict_proba(X_cb)[:, 1]
    return proba

## `compute_best_f1()` Helper Function

In [25]:
def compute_best_f1(probs, targets, thresholds=None):
    probs = np.asarray(probs)
    targets = np.asarray(targets).astype(int)

    if thresholds is None:
        thresholds = np.linspace(0.05, 0.95, 19)

    best_f1, best_t = 0.0, 0.5
    for t in thresholds:
        preds = (probs >= t).astype(int)
        f1 = f1_score(targets, preds, zero_division=0)
        if f1 > best_f1:
            best_f1, best_t = f1, t
    return best_f1, best_t


In [31]:
tabm_train_probs = tabm_predict_proba(df)
xgb_train_probs  = xgb_predict_proba(df)
cb_train_probs   = cb_predict_proba(df)

print("TabM train probs min/mean/max:",
      tabm_train_probs.min(), tabm_train_probs.mean(), tabm_train_probs.max())
print("XGB  train probs min/mean/max:",
      xgb_train_probs.min(), xgb_train_probs.mean(), xgb_train_probs.max())
print("CB   train probs min/mean/max:",
      cb_train_probs.min(), cb_train_probs.mean(), cb_train_probs.max())


Transforming data in 'catboost' mode...
Applying learned statistical transforms (Z-scores)...
CatBoost mode: Skipping target encoding application.
CatBoost mode: Dropping unused object/datetime columns...
Dropping: ['witness_present_ind', 'claim_date']
Transform complete.
TabM train probs min/mean/max: 0.0005119745 0.25603554 0.842113
XGB  train probs min/mean/max: 0.0034326005 0.20450807 0.8821628
CB   train probs min/mean/max: 0.0036373014143064883 0.3419226216986207 0.9449020215461476


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


## Benchmark Models

In [32]:
def model_best_f1(probs, name, thresholds=None):
    if thresholds is None:
        thresholds = np.linspace(0.05, 0.95, 19)
    best_f1, best_t = 0.0, 0.5
    for t in thresholds:
        preds = (probs >= t).astype(int)
        f1 = f1_score(y_train, preds, zero_division=0)
        if f1 > best_f1:
            best_f1, best_t = f1, t
    print(f"{name}: best F1 = {best_f1:.6f} at threshold {best_t:.3f}")
    return best_f1, best_t

In [33]:


# You already computed these earlier:
# tabm_train_probs = tabm_predict_proba(df)
# xgb_train_probs  = xgb_predict_proba(df)
# cb_train_probs   = cb_predict_proba(df)

tabm_f1, tabm_t = model_best_f1(tabm_train_probs, "TabM")
xgb_f1,  xgb_t  = model_best_f1(xgb_train_probs,  "XGB")
cb_f1,   cb_t   = model_best_f1(cb_train_probs,   "CatBoost")

TabM: best F1 = 0.598859 at threshold 0.300
XGB: best F1 = 0.617929 at threshold 0.300
CatBoost: best F1 = 0.608145 at threshold 0.500


## Optuna for Weights

In [34]:
def compute_best_f1(probs, targets, thresholds=None):
    probs = np.asarray(probs)
    targets = np.asarray(targets).astype(int)

    if thresholds is None:
        thresholds = np.linspace(0.05, 0.95, 19)

    best_f1, best_t = 0.0, 0.5
    for t in thresholds:
        preds = (probs >= t).astype(int)
        f1 = f1_score(targets, preds, zero_division=0)
        if f1 > best_f1:
            best_f1, best_t = f1, t
    return best_f1, best_t

In [35]:
def blend3_objective(trial):
    # Raw weights; TabM must be >= 0.5 before normalization
    w_tabm_raw = trial.suggest_float("w_tabm_raw", 0.5, 1.0)
    w_xgb_raw  = trial.suggest_float("w_xgb_raw", 0.0, 1.0)
    w_cb_raw   = trial.suggest_float("w_cb_raw",  0.0, 1.0)

    w_sum = w_tabm_raw + w_xgb_raw + w_cb_raw
    if w_sum == 0:
        w_tabm = w_xgb = w_cb = 1.0 / 3.0
    else:
        w_tabm = w_tabm_raw / w_sum
        w_xgb  = w_xgb_raw  / w_sum
        w_cb   = w_cb_raw   / w_sum

    # Blended probabilities on TRAIN
    blend_probs = (
        w_tabm * tabm_train_probs
        + w_xgb  * xgb_train_probs
        + w_cb   * cb_train_probs
    )

    best_f1, best_t = compute_best_f1(blend_probs, y_train)

    # Log useful info
    trial.set_user_attr("w_tabm", w_tabm)
    trial.set_user_attr("w_xgb",  w_xgb)
    trial.set_user_attr("w_cb",   w_cb)
    trial.set_user_attr("best_threshold", best_t)

    return best_f1

In [36]:
blend3_study = optuna.create_study(
    direction="maximize",
    study_name="tabm_xgb_cb_blend3_tabm_anchor",
)
blend3_study.optimize(blend3_objective, n_trials=150, show_progress_bar=True)

best_trial = blend3_study.best_trial
best_f1 = best_trial.value
best_w_tabm = best_trial.user_attrs["w_tabm"]
best_w_xgb  = best_trial.user_attrs["w_xgb"]
best_w_cb   = best_trial.user_attrs["w_cb"]
best_threshold = best_trial.user_attrs["best_threshold"]

print("\n=== Best 3-model blend on TRAIN ===")
print(f"Best F1 (blend): {best_f1:.6f}")
print(f"Weights: TabM={best_w_tabm:.3f}, XGB={best_w_xgb:.3f}, CB={best_w_cb:.3f}")
print(f"Best threshold: {best_threshold:.4f}")


[I 2025-11-14 12:41:19,434] A new study created in memory with name: tabm_xgb_cb_blend3_tabm_anchor
Best trial: 6. Best value: 0.61344:   5%|▍         | 7/150 [00:00<00:04, 31.32it/s] 

[I 2025-11-14 12:41:19,486] Trial 0 finished with value: 0.6056971514242878 and parameters: {'w_tabm_raw': 0.6689364085102538, 'w_xgb_raw': 0.028669090119382123, 'w_cb_raw': 0.2415865300544281}. Best is trial 0 with value: 0.6056971514242878.
[I 2025-11-14 12:41:19,520] Trial 1 finished with value: 0.6065680730752501 and parameters: {'w_tabm_raw': 0.7135985154581734, 'w_xgb_raw': 0.11869024587449717, 'w_cb_raw': 0.4474983494749931}. Best is trial 1 with value: 0.6065680730752501.
[I 2025-11-14 12:41:19,552] Trial 2 finished with value: 0.6111613876319759 and parameters: {'w_tabm_raw': 0.9147838520387875, 'w_xgb_raw': 0.9793267473313192, 'w_cb_raw': 0.9841745455967549}. Best is trial 2 with value: 0.6111613876319759.
[I 2025-11-14 12:41:19,581] Trial 3 finished with value: 0.6088162456661713 and parameters: {'w_tabm_raw': 0.6315663688648476, 'w_xgb_raw': 0.5805529139673743, 'w_cb_raw': 0.14895641580827512}. Best is trial 2 with value: 0.6111613876319759.
[I 2025-11-14 12:41:19,611] Tria

Best trial: 12. Best value: 0.614428:   8%|▊         | 12/150 [00:00<00:04, 30.67it/s]

[I 2025-11-14 12:41:19,700] Trial 7 finished with value: 0.6132027344148556 and parameters: {'w_tabm_raw': 0.9011443123883018, 'w_xgb_raw': 0.7217640283057078, 'w_cb_raw': 0.0672883746557581}. Best is trial 6 with value: 0.6134401972872996.
[I 2025-11-14 12:41:19,737] Trial 8 finished with value: 0.6094473602963878 and parameters: {'w_tabm_raw': 0.7103040774445017, 'w_xgb_raw': 0.15764285742343698, 'w_cb_raw': 0.8381278478923471}. Best is trial 6 with value: 0.6134401972872996.
[I 2025-11-14 12:41:19,768] Trial 9 finished with value: 0.6112673122982402 and parameters: {'w_tabm_raw': 0.5480362574045676, 'w_xgb_raw': 0.45760306928685524, 'w_cb_raw': 0.9811625105460555}. Best is trial 6 with value: 0.6134401972872996.
[I 2025-11-14 12:41:19,803] Trial 10 finished with value: 0.6114110299566 and parameters: {'w_tabm_raw': 0.501186778341384, 'w_xgb_raw': 0.31115167937166033, 'w_cb_raw': 0.6522359364778957}. Best is trial 6 with value: 0.6134401972872996.
[I 2025-11-14 12:41:19,837] Trial 11

Best trial: 15. Best value: 0.61561:  13%|█▎        | 19/150 [00:00<00:04, 30.35it/s] 

...

[I 2025-11-14 12:41:24,134] Trial 135 finished with value: 0.6130337988261485 and parameters: {'w_tabm_raw': 0.5755309863728444, 'w_xgb_raw': 0.9209413843987693, 'w_cb_raw': 0.7815517340638642}. Best is trial 109 with value: 0.6178163927562023.
[I 2025-11-14 12:41:24,168] Trial 136 finished with value: 0.6172607879924953 and parameters: {'w_tabm_raw': 0.5363409876401355, 'w_xgb_raw': 0.9992132108344414, 'w_cb_raw': 0.07185689775191169}. Best is trial 109 with value: 0.6178163927562023.
[I 2025-11-14 12:41:24,200] Trial 137 finished with value: 0.6156697209924843 and parameters: {'w_tabm_raw': 0.5288471372341972, 'w_xgb_raw': 0.9847631761643778, 'w_cb_raw': 0.11486044673918687}. Best is trial 109 with value: 0.6178163927562023.
[I 2025-11-14 12:41:24,231] Trial 138 finished with value: 0.611066398390342 and parameters: {'w_tabm_raw': 0.6116914764487343, 'w_xgb_raw': 0.4777353143735077, 'w_cb_raw': 0.0745227674012009}. Best is trial 109 with value: 0.6178163927562023.
[I 2025-11-14 12:41

Best trial: 109. Best value: 0.617816:  99%|█████████▉| 149/150 [00:05<00:00, 30.00it/s]

[I 2025-11-14 12:41:24,365] Trial 142 finished with value: 0.6173072883847931 and parameters: {'w_tabm_raw': 0.5347753313489502, 'w_xgb_raw': 0.965790006034672, 'w_cb_raw': 0.040071423514043435}. Best is trial 109 with value: 0.6178163927562023.
[I 2025-11-14 12:41:24,401] Trial 143 finished with value: 0.6176562337527295 and parameters: {'w_tabm_raw': 0.5393823688061273, 'w_xgb_raw': 0.9441190630670475, 'w_cb_raw': 0.07209920482892647}. Best is trial 109 with value: 0.6178163927562023.
[I 2025-11-14 12:41:24,433] Trial 144 finished with value: 0.6146321581414306 and parameters: {'w_tabm_raw': 0.536683713676919, 'w_xgb_raw': 0.5831419949743939, 'w_cb_raw': 0.07166941845532256}. Best is trial 109 with value: 0.6178163927562023.
[I 2025-11-14 12:41:24,467] Trial 145 finished with value: 0.6155752943606693 and parameters: {'w_tabm_raw': 0.5184372827496772, 'w_xgb_raw': 0.943208767642762, 'w_cb_raw': 0.09779908215960478}. Best is trial 109 with value: 0.6178163927562023.
[I 2025-11-14 12:4

Best trial: 109. Best value: 0.617816: 100%|██████████| 150/150 [00:05<00:00, 29.10it/s]

[I 2025-11-14 12:41:24,596] Trial 149 finished with value: 0.6173046670162559 and parameters: {'w_tabm_raw': 0.6033137064820866, 'w_xgb_raw': 0.9373039703266324, 'w_cb_raw': 0.03676039577666043}. Best is trial 109 with value: 0.6178163927562023.

=== Best 3-model blend on TRAIN ===
Best F1 (blend): 0.617816
Weights: TabM=0.345, XGB=0.621, CB=0.034
Best threshold: 0.3000


In [37]:
real_test = pd.read_csv("data/Testing_TriGuard.csv")  # if not already loaded

tabm_test_probs = tabm_predict_proba(real_test)
xgb_test_probs  = xgb_predict_proba(real_test)
cb_test_probs   = cb_predict_proba(real_test)

print("TabM test probs min/mean/max:",
      tabm_test_probs.min(), tabm_test_probs.mean(), tabm_test_probs.max())
print("XGB  test probs min/mean/max:",
      xgb_test_probs.min(), xgb_test_probs.mean(), xgb_test_probs.max())
print("CB   test probs min/mean/max:",
      cb_test_probs.min(), cb_test_probs.mean(), cb_test_probs.max())


Transforming data in 'catboost' mode...
Applying learned statistical transforms (Z-scores)...
CatBoost mode: Skipping target encoding application.
CatBoost mode: Dropping unused object/datetime columns...
Dropping: ['witness_present_ind', 'claim_date']
Transform complete.
TabM test probs min/mean/max: 0.0007758692 0.25367633 0.82720715
XGB  test probs min/mean/max: 0.0035788997 0.20258325 0.8522331
CB   test probs min/mean/max: 0.0035900861756748156 0.3395900545022105 0.9450703883041788


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


In [38]:
blend_test_probs = (
    best_w_tabm * tabm_test_probs
    + best_w_xgb * xgb_test_probs
    + best_w_cb  * cb_test_probs
)

blend_test_preds = (blend_test_probs >= best_threshold).astype(int)


In [39]:
ensemble_pred = pd.DataFrame({
    "claim_number": real_test["claim_number"],
    "subrogation": blend_test_preds,
})

out_path = "results/tabm_xgb_cb_tabmAnchor_optuna_blend.csv"
ensemble_pred.to_csv(out_path, index=False)
print("Saved:", out_path)

Saved: results/tabm_xgb_cb_tabmAnchor_optuna_blend.csv
